In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
import time

In [2]:
ept = pd.read_excel("ept_all.xlsx")
ept.head()

,ISO,Tanggal,Waktu,Ruangan,Pengawas,ID_Peserta,Hadir,NamaPeserta,TempatLahir,TanggalLahir,...,Versi,Kegiatan,Tahun,L1,G1,R1,L2,G2,R2,TotalSkor
0,1,2017-02-01,14.00 - 16.00,C,Nurul Fadhillah,11,True,Rendi,Singkawang,1997-12-05 00:00:00,...,5,-,2017,16.0,17.0,18.0,41.0,40.0,38.0,397.0
1,1,2017-02-01,14.00 - 16.00,C,Nurul Fadhillah,7,True,Muhammad Firdaus,Ketapang,1996-09-04 00:00:00,...,10,-,2017,24.0,12.0,28.0,46.0,35.0,46.0,423.0
2,1,2017-02-01,14.00 - 16.00,C,Nurul Fadhillah,9,True,Yoko Hermanto,Dusun Besar,1995-07-06 00:00:00,...,3,-,2017,12.0,13.0,22.0,37.0,36.0,41.0,380.0
3,1,2017-02-01,14.00 - 16.00,C,Nurul Fadhillah,8,True,Thareq Abdul Aziz,Singkawang,1996-12-23 00:00:00,...,10,-,2017,26.0,22.0,34.0,47.0,45.0,49.0,470.0
4,1,2017-02-01,14.00 - 16.00,C,Nurul Fadhillah,10,True,Afriansyah Akbar,Banjar Baru,1995-04-22 00:00:00,...,4,-,2017,35.0,13.0,26.0,52.0,36.0,45.0,443.0


In [7]:
def appendGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name('key.json', scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    existing_iso_values = worksheet.col_values(worksheet.find('ISO').col)  
    existing_iso_values = existing_iso_values[1:] 
    
    # Convert to set for faster comparison
    existing_iso_set = set(existing_iso_values)

    # Get unique ISO values from the DataFrame
    new_iso_values = set(df['ISO'].astype(str).unique())
    
    # Check for duplicates
    duplicates = existing_iso_set.intersection(new_iso_values)
    if duplicates:
        raise ValueError(f"Duplicate ISO values found: {duplicates}. Aborting the operation.")
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be appended
    data = df.values.tolist()
    
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        worksheet.append_rows(batch, value_input_option='RAW')
        time.sleep(1)

    


    

appendGsheet(df=ept, dari=1, sampai=10)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16604\3489508569.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16604\3489508569.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16604\3489508569.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [6]:
def updateGsheet(df, dari, sampai):
    df = df[(df["ISO"]>=dari) & (df["ISO"]<=sampai)]
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add credentials to the account
    creds = Credentials.from_service_account_file('key.json', scopes=scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Get the instance of the Spreadsheet
    sheet = client.open('qry_ept')

    # Get the first sheet of the Spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    date_columns = ["Tanggal", "TanggalLahir"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
        df[col] = df[col].astype(str)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna('', inplace=True)

    # Data to be update
    data = df.values.tolist()
    
    # Update data in batches to reduce API calls
    batch_size = 100  # Adjust batch size as needed
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        cell_range = f'A{i+15272}:AD{i+15272+len(batch)-1}'  # Adjust the range as needed
        worksheet.update(cell_range, batch)
        time.sleep(1)  # Add delay to avoid hitting rate limits
    
     

# 581 jangan diubah karena di mulai dari cell 15272
updateGsheet(df=ept, dari=581, sampai=621)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22188\4231865365.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col]).dt.strftime('%d-%b-%y')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22188\4231865365.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22188\4231865365.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]